# Extract DayNight Images

Create DayNight database for model training.

In [ ]:
%pip install -qU scikit-learn

In [35]:
import sys
from pathlib import Path
import pandas as pd

current_dir = Path().resolve()
sys.path.append(str(current_dir.parent))

import utils
import os

dataset_folder = Path('../../data/external/o_haze')

In [ ]:
file_list = [str(image.relative_to(dataset_folder)) for folder in dataset_folder.iterdir() if folder.is_dir() for image in folder.iterdir()]
label_list = ['hazy' if 'hazy' in file else 'gt' for file in file_list]

output = {'filename': file_list,
		  'label': label_list}

df_labels = pd.DataFrame(output)
df_labels

In [ ]:
df_count_list = []
for folder in list(dataset_folder.iterdir()):
    df_count = utils.img_size_count(folder, extension='jpg')
    df_count_list.append(df_count)
    
utils.img_size_count_plot(df_count, show=True)

In [ ]:
folders = {
        'hazy': {'label':['hazy']},
        'gt': {'label':['gt']}
    }

for out_folder, df_filter in folders.items():
    img_list = file_list

    output_folder_path = dataset_folder / out_folder
    output_folder_path.mkdir(parents=True, exist_ok=True)
    
    utils.Images.build_dataset(
        img_list=img_list,  # Relative paths
        img_folder=dataset_folder,  # Base folder
        output_folder=dataset_folder / out_folder,  # Output folder
        transformation_function=utils.resize_and_crop,
        transformation_params={'target_size': (256, 256), 'size_filter': [(1600, 1200)]},
        split=0.8,
        random_seed=42
    )